In [1]:
import torch, torchvision
import torch.optim as optim
import torch.nn as nn
import os
#import oil.augLayers as augLayers
from oil.model_trainers.gan import Gan
from oil.model_trainers.classifier import Classifier
from oil.datasetup.datasets import CIFAR10, C10augLayers
from oil.datasetup.dataloaders import getUnlabLoader, getLabLoader
#from oil.architectures.networkparts import layer13,ConvSmallNWN
from oil.architectures import dcgan,resnetgan
from oil.architectures.ganBase import add_spectral_norm, xavier_uniform_init
from oil.utils.utils import cosLr, loader_to,imap,islice
%load_ext autoreload
%autoreload 2

In [2]:
train_epochs = 100
#net_config =        {'numClasses':10}
loader_config =     {'amnt_dev':0,'lab_BS':64,'dataseed':0,'num_workers':2}
#opt_config =        {'lr':.1, 'momentum':.9, 'weight_decay':1e-4, 'nesterov':True}
trainer_config =    {}

trainer_config['log_dir'] = os.path.expanduser('~/tb-experiments/baseline_gan/')
trainer_config['description'] = 'Test being made'


device = torch.device('cuda')
G = resnetgan.Generator().to(device)
D = resnetgan.Discriminator().to(device)
trainset, testset = CIFAR10(False, '~/datasets/cifar10/')

dataloaders = {}
dataloaders['train'], _ = getLabLoader(trainset,**loader_config)
dataloaders['dev'] = islice(dataloaders['train'],4096//loader_config['lab_BS'])
dataloaders = {k: imap(lambda z:z[0].to(device),v) for k,v in dataloaders.items()}

trainer = Gan(G,dataloaders,D=D,n_disc=5, **trainer_config)


Files already downloaded and verified
Files already downloaded and verified
Creating Train, Dev split         with 50000 Train and 0 Dev


In [ ]:
trainer.train(train_epochs)
trainer.save_checkpoint()

/home/marc/anaconda3/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")
/home/marc/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1961: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f625e469fd0>>
Traceback (most recent call last):
  File "/home/marc/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 399, in __del__
    self._shutdown_workers()
  File "/home/marc/anaconda3/lib/python3.6/site-packages/torch/utils

       G_loss    D_loss          FID        IS
783 -0.251582  2.027959  1906.193428  1.436664
